In [1]:
import pandas as pd
import numpy as np
import os as os 

from matplotlib import gridspec
import matplotlib.pyplot as plt
%matplotlib inline  

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.discrete.conditional_models import ConditionalLogit

from IPython.display import display    


import scipy.stats 

from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso, Ridge, LassoCV, LogisticRegressionCV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.metrics import mean_squared_error

In [6]:
!pip install stnomics


ERROR: Could not find a version that satisfies the requirement stnomics (from versions: none)
ERROR: No matching distribution found for stnomics


In [5]:
import stnomics as st

ModuleNotFoundError: No module named 'stnomics'

### Bring in simulated Data

In [7]:
def generate_data():
    N = 2000
    
    cov = [[1.00, 0.08, 0.05, 0.05],
           [0.08, 1.00,-0.08,-0.02],
           [0.05,-0.08, 1.00,-0.10],
           [0.05,-0.02,-0.10, 1.00]]
    cov = np.eye(4)
    X = np.random.multivariate_normal(np.zeros(4), cov,N)
    x1,x2,x3,x4= X[:,0],X[:,1],X[:,2],X[:,3]

    treatment_latent = 2*np.sin( np.pi * x4 * x3) + 10*(x2-0.5)**2 - 10*x1
    m,s = np.average(treatment_latent), np.std(treatment_latent)

    treatment_latent = (treatment_latent - m) / s
    
    random_t = np.random.normal(0,1,N)
    
    treatment_latent += random_t
    
    treatment = np.array( np.exp(treatment_latent) / (1+ np.exp(treatment_latent)) > np.random.uniform(0,1,N) ).astype(np.int32)

#     Y = 100 +0.5*x1 - 6*x2 + -2*x4*x1 + 0.5*x1*x2 - 7*(x3+1)**(0.5) + 8/(0.5+x3+x4)
    Y = 100 + 10*np.sin( np.pi * x1 * x2) + 20*(x3-0.5)**2 - 10*x4
#     GT = np.std(Y)
    random_y = np.random.normal(0,1,N)

    GT = 5
    Y += np.random.normal(1,2,N)
    Y += GT*(treatment==1) 
    
    df_est = pd.DataFrame({'x1':x1, 'x2':x2,'x3':x3,'x4':x4,'treatment':treatment, 'Y':Y, 'GT':GT} )
    df_est['x1_2'] = df_est['x1'].pow(2)
    df_est['x2_2'] = df_est['x2'].pow(2)
    df_est['x3_2'] = df_est['x3'].pow(2)
    df_est['x4_2'] = df_est['x4'].pow(2)    
    return df_est

In [8]:
generate_data()

,x1,x2,x3,x4,treatment,Y,GT,x1_2,x2_2,x3_2,x4_2
0,0.711549,0.622420,-1.585916,-1.538460,0,215.418054,5,0.506302,0.387407,2.515129,2.366859
1,1.925178,2.309328,0.499108,-1.570371,0,127.482153,5,3.706310,5.332996,0.249109,2.466066
2,1.056995,-0.913222,-0.723167,-0.642418,0,136.974960,5,1.117238,0.833974,0.522970,0.412700
3,0.397976,1.117132,0.799746,1.409213,0,97.500567,5,0.158385,1.247984,0.639594,1.985881
4,-2.873839,-0.940221,-1.223259,0.961693,1,160.874562,5,8.258949,0.884016,1.496362,0.924854
...,...,...,...,...,...,...,...,...,...,...,...
1995,2.394090,0.043904,-1.275395,0.801911,0,157.666272,5,5.731665,0.001928,1.626631,0.643061
1996,-0.754822,0.442279,-1.207770,1.060237,0,141.098258,5,0.569756,0.195611,1.458708,1.124102
1997,1.192743,0.668187,0.742945,-0.661910,0,112.483936,5,1.422636,0.446473,0.551967,0.438125
1998,-0.493963,-2.878483,1.010368,0.488118,1,100.011418,5,0.243999,8.285666,1.020843,0.238259


In [9]:
model_max_iter = 500
## treatment prediction models
t_models = {}
t_models['LogitCV'] = LogisticRegressionCV(cv=5, random_state=27, n_jobs=-1)
t_models['logit'] = LogisticRegression(penalty='l2',solver='lbfgs', C=1, max_iter=model_max_iter, fit_intercept=True)
t_models['logit_L1_C2'] = LogisticRegression(penalty='l1',C=2, max_iter=model_max_iter, fit_intercept=True)
t_models['logit_L2_C5'] = LogisticRegression(penalty='l2',C=2, max_iter=model_max_iter, fit_intercept=True)
t_models['rf_md10'] = RandomForestClassifier(n_estimators=25,max_depth=10, min_samples_split=200,n_jobs=-1)
t_models['rf_md3'] = RandomForestClassifier(n_estimators=25,max_depth=3, min_samples_split=200,n_jobs=-1)
t_models['nn'] = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(3, 2), random_state=1,max_iter=model_max_iter)
## outcome prediction models
y_models = {}
y_models['LassoCV'] = LassoCV(cv=5, n_jobs=-1,  random_state=27)
y_models['ols'] = LinearRegression()
y_models['lasso_a2'] = Lasso(alpha=2,max_iter=model_max_iter)
y_models['ridge_a2'] = Ridge(alpha=2,max_iter=model_max_iter)
y_models['rf_md10'] = RandomForestRegressor(n_estimators=25,max_depth=10, min_samples_split=200,n_jobs=-1)
y_models['rf_md3'] = RandomForestRegressor(n_estimators=25,max_depth=3, min_samples_split=200,n_jobs=-1)
y_models['nn'] = MLPRegressor(alpha=1e-5, hidden_layer_sizes=(3, 2), random_state=1, max_iter=model_max_iter)

In [10]:
n_data_splits = 4
aux_dictionary = {'n_bins': 2, 'n_trees':2, 'max_depth':2, 
                  'upper':0.999, 'lower':0.001,
                  'bootstrapreps':100,
                  'subsample_ratio':0.5}
bootstrap_number = 5

In [11]:
df = generate_data()

feature_list = [x for x in df.columns if 'x' in x]

ols = st.ate.ols_vanilla(df, 
                'splits', feature_list, 'Y', 'treatment',
                y_models['LassoCV'],t_models['LogitCV'],
               n_data_splits, aux_dictionary )
pbin = st.ate.propbinning(df, 
                'splits', feature_list, 'Y', 'treatment',
                y_models['LassoCV'],t_models['LogitCV'],
               n_data_splits, aux_dictionary )
plm = st.ate.dml.dml_plm(df, 
                'splits', feature_list, 'Y', 'treatment',
                y_models['LassoCV'],t_models['LogitCV'],
               n_data_splits, aux_dictionary )
irm = st.ate.dml.dml_irm(df, 
                'splits', feature_list, 'Y', 'treatment',
                y_models['LassoCV'],t_models['LogitCV'],
               n_data_splits, aux_dictionary )
ip = st.ate.ipw(df, 
                'splits', feature_list, 'Y', 'treatment',
                y_models['LassoCV'],t_models['LogitCV'],
               n_data_splits, aux_dictionary )

NameError: name 'st' is not defined